In [17]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from time import sleep
import quandl 
import os
quandl.ApiConfig.api_key = os.getenv("QUANDL_API_KEY")
%matplotlib inline

In [18]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [19]:
# Get all Assets
assets = api.list_assets()

# Keep only tradeable assets
tradeable = [asset for asset in assets if asset.tradable ]

In [20]:
# Create a new empty DataFrame
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets if asset.exchange=="NASDAQ" or asset.exchange=="NYSE"])

# Display the first 10 asset tickers
display(asset_info_df.head(10))

,symbol
0,ADTX
1,BAC
2,DYFN
3,AAME
4,ACIU
5,ACV
6,ADXN
7,ADXS
8,AERI
9,AES


In [21]:
print(asset_info_df.count())

symbol    7109
dtype: int64


In [22]:
print(9783/180)

54.35


In [23]:
asset_info = [asset.symbol for asset in assets]

In [57]:
# Set timeframe to '1D'
timeframe = '1D'
start_date = pd.Timestamp("2018-07-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-06-23", tz="America/New_York").isoformat()

stockprices = []

i = 0
for asset in asset_info:
    print(asset)
    df = api.get_barset(
        asset,
        timeframe,
        limit=None,
        start=start_date,
        end=end_date,
        after=None,
        until=None,
    ).df


    
    dfq = quandl.get_table('SHARADAR/DAILY', ticker=asset)
    #print(df)
    if df.empty:
        print("     - Empty - Skip")
    elif dfq.empty:
        print("     - Empty - Skip")
    else:
        # format alpaca df for joining
        df = df.stack(level=0)
        df = df.rename_axis(('date', 'ticker'))
        df = df.reset_index()
        df['date'] = df['date'].dt.date
        df = df.set_index(["date", "ticker"])
        
        dfq = dfq.set_index(["date", "ticker"])
        dfb = df.join(dfq)
        print(dfb.head(2))
        #stockprices.append({"Ticker" : asset, "Metrics": dfb})
        stockprices.append(dfb)
        # print("Length:", len(stockprices))
        i += 1
        sleep(1)
        
        # take these two lines out when you're ready to run for all assets
        if i > 3:
            break

ADTX
     - Empty - Skip
ALTL
     - Empty - Skip
BAC
                     close   high    low   open    volume lastupdated  \
date       ticker                                                       
2018-07-02 BAC     28.2700  28.28  27.83  28.08  47131743  2018-10-21   
2018-07-03 BAC     27.8053  28.43  27.74  28.33  37068322  2018-10-21   

                         ev  evebit  evebitda  marketcap   pb    pe   ps  
date       ticker                                                         
2018-07-02 BAC     278657.9     9.0       8.4   286740.9  1.1  15.3  3.2  
2018-07-03 BAC     273588.3     8.8       8.3   281671.3  1.1  15.0  3.2  
DYFN
     - Empty - Skip
AAME
                   close  high   low  open  volume lastupdated    ev  evebit  \
date       ticker                                                              
2018-07-02 AAME     2.60  2.75  2.40  2.75    9781  2018-10-21  77.0    68.3   
2018-07-03 AAME     2.55  2.65  2.45  2.55    1554  2018-10-21  77.0    68.3   

  

In [58]:
pd.concat(stockprices)

,,close,high,low,open,volume,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps
date,ticker,,,,,,,,,,,,,
2018-07-02,BAC,28.2700,28.2800,27.8300,28.0800,47131743,2018-10-21,278657.9,9.0,8.4,286740.9,1.1,15.3,3.2
2018-07-03,BAC,27.8053,28.4300,27.7400,28.3300,37068322,2018-10-21,273588.3,8.8,8.3,281671.3,1.1,15.0,3.2
2018-07-05,BAC,27.9250,28.0790,27.8100,27.9500,40145714,2018-10-21,275007.8,8.9,8.3,283090.8,1.1,15.1,3.2
2018-07-06,BAC,28.0350,28.1500,27.6300,27.8100,34269892,2018-10-21,276123.1,8.9,8.3,284206.1,1.1,15.2,3.2
2018-07-09,BAC,29.0650,29.0900,28.2200,28.2300,55470499,2018-10-21,286465.2,9.2,8.6,294548.2,1.1,15.7,3.3
2018-07-10,BAC,28.8250,29.3300,28.7500,29.2200,53604698,2018-10-21,284234.6,9.2,8.6,292317.6,1.1,15.6,3.3
2018-07-11,BAC,28.6700,28.9100,28.6200,28.6700,42555810,2018-10-21,282713.7,9.1,8.5,290796.7,1.1,15.5,3.3
2018-07-12,BAC,28.7700,28.9700,28.6600,28.9000,44388039,2018-10-21,283626.2,9.1,8.6,291709.2,1.1,15.6,3.3
2018-07-13,BAC,28.5400,28.6900,28.2223,28.6200,67531865,2018-10-21,281395.6,9.1,8.5,289478.6,1.1,15.4,3.3


In [ ]:
# df = pd.DataFrame({'Index': [] })

In [51]:
# pd.concat([df, dfq], join='inner', ignore_index=False)

In [30]:
print(api.data_get(path, data=None))

NameError: name 'path' is not defined

In [44]:
print(len(stockprices))


19


In [45]:
print(stockprices[0])

                             MNE                                  
                            open     high      low    close volume
2018-07-02 00:00:00-04:00  12.61  12.7100  12.6100  12.6640   8736
2018-07-03 00:00:00-04:00  12.65  12.7200  12.6500  12.7183   6476
2018-07-05 00:00:00-04:00  12.73  12.7700  12.7000  12.7400   1653
2018-07-06 00:00:00-04:00  12.71  12.7600  12.7100  12.7400   2310
2018-07-09 00:00:00-04:00  12.74  12.7400  12.7001  12.7001   4575
...                          ...      ...      ...      ...    ...
2020-06-09 00:00:00-04:00  14.60  14.6000  14.5600  14.5600  11939
2020-06-10 00:00:00-04:00  14.56  14.6100  14.5600  14.5700    800
2020-06-11 00:00:00-04:00  14.66  14.6600  14.5700  14.5900   3570
2020-06-12 00:00:00-04:00  14.66  14.6600  14.6330  14.6330   1396
2020-06-15 00:00:00-04:00  14.61  14.6834  14.5500  14.5500  47540

[486 rows x 5 columns]
